In [ ]:
#for using of yolo
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/107.7 GB disk)


In [ ]:
#mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd ../

/


In [ ]:
%ls

bin@                        datalab/  kaggle/  libx32@                   opt/   sbin@  tools/
boot/                       dev/      lib@     media/                    proc/  srv/   usr/
content/                    etc/      lib32@   mnt/                      root/  sys/   var/
cuda-keyring_1.0-1_all.deb  home/     lib64@   NGC-DL-CONTAINER-LICENSE  run/   tmp/


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.6 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size

In [ ]:
image = '/content/input_image.jpg'

In [ ]:
import os
import re
import cv2
import subprocess
from PIL import Image
from ultralytics import YOLO

In [ ]:
import gradio as gr

def predict(image):
        input_path = image # Assuming image is a file-like object
        # Execute the YOLO command to perform object detection and get the output folder path
        yolo_command = "yolo task=detect mode=predict model=/content/drive/MyDrive/License\ plate\ detection\ project/runs/detect/25epoch_results/weights/best.pt conf=0.25 source='" + input_path + "' hide_labels=True line_thickness=1 "
        output = subprocess.check_output(yolo_command, shell=True).decode("utf-8")
        output_lines = output.strip().split('\n')
        output_folder = None
        for line in output_lines:
            if line.startswith("Results saved to "):
                output_folder = re.sub(r'\x1b\[[0-9;]*m', '', line[len("Results saved to "):].strip())  # Remove ANSI escape codes
                break
        if output_folder is None:
            raise ValueError("Output folder path not found in the command output.")

        print("Output folder:", output_folder)  # Print the output folder path for debugging

        # Find the image file within the output folder
        image_files = [f for f in os.listdir(output_folder) if f.endswith('.jpg') or f.endswith('.png') or f.endswith('.JPG') or f.endswith('.jpeg')]
        if not image_files:
            raise FileNotFoundError("No image files found in the output folder.")

        # Assuming there's only one image file in the folder, select the first one
        image_file_path = os.path.join(output_folder, image_files[0])

        # Open the image
        #img = Image.open(image_file_path)

        # Get width and height
        #width = img.width
        #height = img.height

        # Load the YOLOv8 model
        model = YOLO('/content/drive/MyDrive/License plate detection project/runs/detect/25epoch_results/weights/best.pt')

        # Perform inference on the image
        results = model(image_file_path)

        # Extract bounding boxes, classes, names, and confidences
        boxes = results[0].boxes.xyxy.tolist()
        classes = results[0].boxes.cls.tolist()
        names = results[0].names
        confidences = results[0].boxes.conf.tolist()

        # Create a folder to store the results
        output_folder = "results"
        os.makedirs(output_folder, exist_ok=True)

        # Create a text file to store all the results
        output_file = os.path.join(output_folder, "all_results.txt")
        with open(output_file, "w") as f:
            # Iterate through the results
            x=[]
            for i, (box, cls, conf) in enumerate(zip(boxes, classes, confidences)):
                x.append(box)
                #x1, y1, x2, y2 = box
                confidence = conf
                detected_class = cls
                name = names[int(cls)]

                # Write the result to the file
                f.write(f"BBox: {x[i][0]},{x[i][1]},{x[i][2]},{x[i][3]}\n")

        print(f"All results stored in {output_file}")

        import numpy as np
        from PIL import Image
        import matplotlib.pyplot as plt

        # Load the image as a NumPy array
        image_array = np.array(Image.open(image_file_path))

        for i in range(len(x)):

            x1, x2 = min(x[i][0], x[i][2]), max(x[i][0], x[i][2])
            y1, y2 = min(x[i][1], x[i][3]), max(x[i][1], x[i][3])

            # Crop the image array using the floating-point coordinates
            cropped_array = image_array[int(y1):int(y2), int(x1):int(x2)]

            # Convert the cropped array back to an image
            cropped_image = Image.fromarray(cropped_array)

            # Save the cropped image with a unique filename incorporating the index i
            cropped_image.save(f"cropped_image_{i}.jpg")
            np.save(f"cropped_array_{i}.npy", cropped_array)

            # Display the cropped image
            plt.imshow(cropped_image)
            plt.show()

            import cv2
        # Match contours to license plate or character template
        def find_contours(dimensions, img) :
            plt.imshow(img)
            plt.show()
            # Find all contours in the image

            cntrs, _ = cv2.findContours(img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            # Retrieve potential dimensions
            lower_width = dimensions[0]
            upper_width = dimensions[1]
            lower_height = dimensions[2]
            upper_height = dimensions[3]

            # Check largest 5 or  15 contours for license plate or character respectively
            cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]

            ii = cv2.imread('contour.jpg')
            # Set the border pixels to white
            border_size = 5
            ii[:border_size+5, :] = 255  # Top border
            ii[-border_size:, :] = 255  # Bottom border
            ii[:, :border_size] = 255  # Left border
            ii[:, -border_size:] = 255  # Right border
            plt.imshow(ii)
            plt.show()
            x_cntr_list = []
            target_contours = []
            img_res = []
            for cntr in cntrs :
                # detects contour in binary image and returns the coordinates of rectangle enclosing it
                intX, intY, intWidth, intHeight = cv2.boundingRect(cntr)

                # checking the dimensions of the contour to filter out the characters by contour's size
                if intWidth > lower_width and intWidth < upper_width and intHeight > lower_height and intHeight < upper_height :
                    x_cntr_list.append(intX) #stores the x coordinate of the character's contour, to used later for indexing the contours

                    char_copy = np.zeros((44,24))
                    # extracting each character using the enclosing rectangle's coordinates.
                    char = img[intY:intY+intHeight, intX:intX+intWidth]
                    char = cv2.resize(char, (20, 40))

                    cv2.rectangle(ii, (intX,intY), (intWidth+intX, intY+intHeight), (50,21,200), 2)
                    plt.imshow(ii, cmap='gray')
                    plt.title('Predict Segments')

                    # Make result formatted for classification: invert colors
                    char = cv2.subtract(255, char)

                    # Resize the image to 24x44 with black border
                    char_copy[2:42, 2:22] = char
                    char_copy[0:2, :] = 0
                    char_copy[:, 0:2] = 0
                    char_copy[42:44, :] = 0
                    char_copy[:, 22:24] = 0

                    img_res.append(char_copy) # List that stores the character's binary image (unsorted)

            # Return characters on ascending order with respect to the x-coordinate (most-left character first)

            plt.show()
            # arbitrary function that stores sorted list of character indeces
            indices = sorted(range(len(x_cntr_list)), key=lambda k: x_cntr_list[k])
            img_res_copy = []
            for idx in indices:
                img_res_copy.append(img_res[idx])# stores character images according to their index
            img_res = np.array(img_res_copy)

            return img_res
        # Find characters in the resulting images
        def segment_characters(image) :

            # Preprocess cropped license plate image
            img_lp = cv2.resize(image, (333, 75))
            img_gray_lp = cv2.cvtColor(img_lp, cv2.COLOR_BGR2GRAY)
            _, img_binary_lp = cv2.threshold(img_gray_lp, 200, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            img_binary_lp = cv2.erode(img_binary_lp, (3,3))
            img_binary_lp = cv2.dilate(img_binary_lp, (3,3))

            LP_WIDTH = img_binary_lp.shape[0]
            LP_HEIGHT = img_binary_lp.shape[1]

            # Make borders white
            img_binary_lp[0:3,:] = 255
            img_binary_lp[:,0:3] = 255
            img_binary_lp[72:75,:] = 255
            img_binary_lp[:,330:333] = 255

            # Estimations of character contours sizes of cropped license plates
            dimensions = [LP_WIDTH/6,
                              LP_WIDTH/2,
                              LP_HEIGHT/10,
                              2*LP_HEIGHT/3]
            plt.imshow(img_binary_lp, cmap='gray')
            plt.title('Contour')
            plt.show()
            cv2.imwrite('contour.jpg',img_binary_lp)

            # Get contours within cropped license plate
            char_list = find_contours(dimensions, img_binary_lp)

            return char_list
        cropped_array = np.load("cropped_array_0.npy")
        char=segment_characters(cropped_array)
        from math import sqrt
        from sklearn.preprocessing import MinMaxScaler

        def ReLU(X):
            """ReLU function"""
            return np.maximum(0,X)

        def softmax(X):
            """Softmax function"""
            e_X = np.exp(X - np.max(X))
            return e_X / e_X.sum(axis=0)

        def deriv_ReLU(X):
            """Derivative of ReLU function"""
            return X > 0

        def one_hot_encoder(y):
            """One Hot Encoder"""
            oh_y = np.zeros((y.size, y.max() + 1))
            oh_y[np.arange(y.size), y] = 1
            return oh_y.T

        def get_predictions(A2):
            return np.argmax(A2, axis=0)

        def get_accuracy(pred, y):
            #print(pred, y)
            return np.sum(pred == y) / y.size

        def model_pred(W1, b1, W2, b2, X):
            """Forward propagation steps"""
            Z1 = W1.dot(X) + b1
            A1 = ReLU(Z1)
            Z2 = W2.dot(A1) + b2
            A2 = softmax(Z2)
            pred = get_predictions(A2)
            return pred
        # Load weights and biases from files
        W1 = np.load('/content/drive/MyDrive/License plate detection project/weights_cnn/w1.npy')
        W2 = np.load('/content/drive/MyDrive/License plate detection project/weights_cnn/w2.npy')
        b1 = np.load('/content/drive/MyDrive/License plate detection project/weights_cnn/bias1.npy')
        b2 = np.load('/content/drive/MyDrive/License plate detection project/weights_cnn/bias2.npy')


        #W1 = np.load('/content/drive/MyDrive/License plate detection project/letter dataset/w1.npy')
        #W2 = np.load('/content/drive/MyDrive/License plate detection project/letter dataset/w2.npy')
        #b1 = np.load('/content/drive/MyDrive/License plate detection project/letter dataset/b1.npy')
        #b2 = np.load('/content/drive/MyDrive/License plate detection project/letter dataset/b2.npy')

          # Define the width and height for resizing
        target_shape = (28, 28)
        # Loop through the images in 'char'
        output=''
        for i, image_data in enumerate(char):
            # Resize the image using cv2.resize()
            img_resized = cv2.resize(image_data, target_shape)  # Resize image

            # Flatten image into 1D array
            img_flattened = img_resized.flatten()  # Flatten image into 1D array

            img_flattened = img_flattened.reshape(-1, 1)

            # Perform predictions using the model
            test_pred = model_pred(W1, b1, W2, b2, img_flattened)
            #print(type(test_pred))
            my_char=None
            if(test_pred[0]>9):
              my_char = chr(ord('A') + test_pred[0] - 10)
            else:
              my_char=str(test_pred[0])
            output+=my_char


            # Display the resized image using cv2.imshow()
            #plt.imshow(img_resized, cmap='gray')  # Display the image
            #plt.axis('off')  # Turn off axis labels
            #plt.show()
            # Print predictions and length of flattened image
            #print(my_char )

        print("\nfinal predicted numer plate :",output)
        return output

#out=predict(image)
#print("hiiii",out)
iface = gr.Interface(fn=predict, inputs="file", outputs="text", title="License Plate Detection", description="Upload an image containing a license plate to detect the characters on the plate.")
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ece036a779e9fda069.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Output folder: runs/detect/predict

image 1/1 /content/runs/detect/predict/cropped_parking_lot_283.JPG: 128x224 1 number plate, 47.4ms
Speed: 1.1ms preprocess, 47.4ms inference, 1616.4ms postprocess per image at shape (1, 3, 128, 224)
All results stored in results/all_results.txt

final predicted numer plate : YAC3B06
Output folder: runs/detect/predict2

image 1/1 /content/runs/detect/predict2/cropped_parking_lot_248.JPG: 128x224 1 number plate, 40.6ms
Speed: 1.0ms preprocess, 40.6ms inference, 1.3ms postprocess per image at shape (1, 3, 128, 224)
All results stored in results/all_results.txt

final predicted numer plate : BZY8Z87
Output folder: runs/detect/predict3

image 1/1 /content/runs/detect/predict3/monitoring_system_63.JPG: 192x224 1 number plate, 62.5ms
Speed: 1.8ms preprocess, 62.5ms inference, 2.8ms postprocess per image at shape (1, 3, 192, 224)
All results stored in results/all_results.txt

final predicted numer plate : AIRIA48
Output folder: runs/detect/predict4

image 1/

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1786, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1338, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r

Output folder: runs/detect/predict7

image 1/1 /content/runs/detect/predict7/parking_lot1_4.JPG: 128x224 1 number plate, 24.1ms
Speed: 0.8ms preprocess, 24.1ms inference, 0.8ms postprocess per image at shape (1, 3, 128, 224)
All results stored in results/all_results.txt

final predicted numer plate : RZZ8A53
Output folder: runs/detect/predict8

image 1/1 /content/runs/detect/predict8/test3.jpg: 224x224 2 number plates, 69.0ms
Speed: 2.3ms preprocess, 69.0ms inference, 3.9ms postprocess per image at shape (1, 3, 224, 224)
All results stored in results/all_results.txt

final predicted numer plate : KL01KLKL01


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1786, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1338, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    r

Output folder: runs/detect/predict9

image 1/1 /content/runs/detect/predict9/test-1.JPG: 128x224 1 number plate, 28.6ms
Speed: 0.7ms preprocess, 28.6ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 224)
All results stored in results/all_results.txt

final predicted numer plate : BZY8Z87
